#https://mmorley.hatenablog.com/entry/fx_backtesting01

Start                     2019-09-30 00:01:00 ヒストリカルデータの開始日時

End                       2019-09-30 23:59:00 ヒストリカルデータの終了日時

Duration                      0 days 23:58:00 ヒストリカルデータの期間

Exposure [%]                          92.7677 ポジションを持っていた期間の割合（ポジションを持っていた期間÷全期間×100）

**Equity Final [$]                      9715.39 所持金の最終値**

**Equity Peak [$]                       10006.7 所持金の最高値**

**Return [%]                           -2.84615 利益率=損益÷開始時所持金×100**

Buy & Hold Return [%]                0.101896 （（終了時の終値 - 開始時の終値）÷ 開始時の終値）の絶対値×100

**Max. Drawdown [%]                    -2.94051 最大下落率**

**Avg. Drawdown [%]                  -0.0155023 平均下落率**

Max. Drawdown Duration        0 days 00:30:00 最大下落期間

Avg. Drawdown Duration        0 days 00:10:00 平均下落期間

**Trades                                   54 取引回数**

**Win Rate [%]                          9.25926 勝率=勝ち取引回数÷全取引回数×100**

Best Trade [%]                      0.0639561 1回の取引での利益の最大値÷所持金×100

Worst Trade [%]                     -0.149003 1回の取引での損失の最大値÷所持金×100

Avg. Trade [%]                     -0.0529873 損益の平均値÷所持金×100

Max. Trade Duration           0 days 01:50:00 1回の取引での最長期間

Avg. Trade Duration           0 days 00:25:00 1回の取引での平均期間

**Expectancy [%]                      0.0566947 期待値=平均利益×勝率＋平均損失×敗率**

期待値とは、１回の取引で期待できる利益のことです。

売買ルールの優劣を判断する指標で、勝率、平均利益、平均損失から求めます。

期待値が正の場合は資産が増えていき、負の場合は資産が減っていくと考えます。

**SQN                                  -10.6372 # SQN（SystemQualityNumber）**

SQNとは、Van K. Tharp氏によって定義された取引システムの分類（評価）方法です。

SQN値によって下記のように分類されます。

1.6-1.9：平均以下

2.0-2.4：平均

2.5-2.9：良い

3.0-5.0：素晴らしい

5.1-6.9：最高

7.0～：聖杯？

取引数が30以上の場合、SQN値は信頼できるとみなされるでしょう。

Sharpe Ratio                         -1.45463 標準偏差に対するリターンの比率

シャープレシオとは利益とリスクの比率のことで、値が大きいほど資産曲線がなめらかになり安定性のある利益が見込めます。

Sortino Ratio                        -1.87251 下方リスクに対するリターンの比率

シャープレシオだけでは分からない下方リスクの抑制度合いを判断する場合に使われます。

通常、この数値が大きいほど優れている（下落局面に強い）ことを示します。

Calmar Ratio                       -0.0180198 最大損失率に対する年間平均収益の比率

値が低いほど指定された期間に渡ってリスク調整ベースで実行された投資は悪化します。

値が高いほどパフォーマンスが向上します。

_strategy                          myStrategy 使用したストラテジーの関数名

In [1]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import talib as ta
import importlib
import os
import platform
import psutil

In [2]:
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('AAPL', 'yahoo', start, end)
data = data.astype("double")
print(data)
loss_cut = true

initial = 1000000

               High      Low     Open    Close     Volume  Adj Close
Date                                                                
2021-09-13  17510.0  17240.0  17400.0  17510.0  8217909.0    17510.0
2021-09-14  17920.0  17570.0  17660.0  17760.0  9275477.0    17760.0
2021-09-15  17670.0  17390.0  17520.0  17570.0  7529060.0    17570.0
2021-09-16  17720.0  17230.0  17710.0  17360.0  8158329.0    17360.0
2021-09-17  17620.0  17400.0  17450.0  17570.0  5910472.0    17570.0
...             ...      ...      ...      ...        ...        ...
2022-09-06  14425.0  14160.0  14275.0  14215.0  6700517.0    14215.0
2022-09-07  14155.0  13860.0  14140.0  14045.0  9128578.0    14045.0
2022-09-08  14700.0  14340.0  14345.0  14685.0  9606072.0    14685.0
2022-09-09  14925.0  14710.0  14850.0  14865.0  9063140.0    14865.0
2022-09-12  15270.0  15080.0  15165.0  15155.0  6206979.0    15155.0

[245 rows x 6 columns]


**MACD CROSS**

In [3]:
import lib.BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot() # 実行結果（グラフ）

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    41.22449
Equity Final [$]                 976107.69675
Equity Peak [$]                   1043735.936
Return [%]                           -2.38923
Buy & Hold Return [%]              -13.449457
Return (Ann.) [%]                   -2.456649
Volatility (Ann.) [%]               23.124661
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.965835
Avg. Drawdown [%]                   -9.524451
Max. Drawdown Duration      250 days 00:00:00
Avg. Drawdown Duration       91 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      12.627233
Worst Trade [%]                     -7.587629
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,64,61,79,15556.62600,15025.0,-34024.06400,-0.034174,2021-12-13,2022-01-11,29 days
1,68,97,108,14164.77025,13090.0,-73084.37700,-0.075876,2022-02-04,2022-02-22,18 days
2,67,124,139,13305.53800,13780.0,31788.95400,0.035659,2022-03-17,2022-04-08,22 days
3,67,163,182,13687.41900,13305.0,-25622.07300,-0.027939,2022-05-18,2022-06-14,27 days
4,66,193,194,13526.62700,13040.0,-32117.38200,-0.035975,2022-06-29,2022-06-30,1 days
5,65,200,231,13300.51325,14980.0,109166.63875,0.126272,2022-07-08,2022-08-24,47 days


Row(id='1492', ...)

In [4]:
#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output2=bt.optimize(n1=range(10, 100, 5),n2=range(10, 200, 10),ns=range(10, 50, 5), maximize='Equity Final [$]', method='grid')
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 2736 configurations.
  output = _optimize_grid()
c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   16.734694
Equity Final [$]                1126381.05625
Equity Peak [$]                 1206256.05625
Return [%]                          12.638106
Buy & Hold Return [%]              -13.449457
Return (Ann.) [%]                   13.021759
Volatility (Ann.) [%]               15.109648
Sharpe Ratio                         0.861817
Sortino Ratio                        1.608207
Calmar Ratio                         1.111058
Max. Drawdown [%]                  -11.720148
Avg. Drawdown [%]                   -3.889238
Max. Drawdown Duration       26 days 00:00:00
Avg. Drawdown Duration       10 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      12.785213
Worst Trade [%]                     12.785213
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,75,203,243,13179.91925,14865.0,126381.05625,0.127852,2022-07-13,2022-09-09,58 days


<Strategy MACDCross(n1=35,n2=180,ns=15)>

Row(id='2714', ...)

**EMA CROSS**

In [ ]:
import lib.BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot() # 実行結果（グラフ）

In [ ]:
#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    n1=range(10, 100, 10),\
    n2=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid')
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

**SMA**

In [ ]:
import lib.BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot() # 実行結果（グラフ）

In [ ]:
# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    n1=range(10, 100, 5),\
    n2=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid')
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

**ボリンジャーバンド**

In [ ]:
import lib.BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot() # 実行結果（グラフ）